In [3]:
from typing import List
def get_requirements()-> List:
    """

    This function returns a list of requirements in list format
    """
    try:
        requirement_list:List[str] = []

        with open('requirements.txt', 'r') as f:
            lines = f.readlines()
        requirement_list = [line.replace('\n', '') for line in lines]
        if "-e ." in requirement_list:
            requirement_list.remove("-e .")

        return requirement_list
    except Exception as e:
        raise e

In [4]:
get_requirements()

['pymongo[srv]==4.2.0']

In [1]:
import pandas as pd


In [4]:
train_file_path = r"C:\ineuron\Industry_ready_projects\Sensor_fault_detection\Scania_APS_Fault_detection\artifact\10_30_2022_00_18_29\data_ingestion\ingested\train.csv"

df = pd.read_csv(train_file_path)
df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,54,4.0,36.0,30.0,0.0,0.0,0.0,0.0,0.0,...,1224.0,176.0,216.0,74.0,36.0,108.0,0.0,0.0,0.0,0.0
1,neg,38876,NaN,744.0,596.0,0.0,0.0,0.0,0.0,0.0,...,576452.0,221750.0,315260.0,192402.0,104948.0,66270.0,117258.0,1684.0,0.0,0.0
2,neg,15654,NaN,42.0,14.0,0.0,0.0,0.0,0.0,0.0,...,41490.0,17120.0,52632.0,110184.0,67478.0,357888.0,3386.0,0.0,0.0,0.0
3,neg,1002,NaN,16.0,18.0,0.0,0.0,0.0,0.0,0.0,...,7110.0,3012.0,7170.0,4956.0,12754.0,9538.0,0.0,0.0,0.0,0.0
4,neg,4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,10.0,14.0,12.0,22.0,38.0,0.0,0.0,0.0,0.0


In [29]:
cols = df.columns
zero_std=[]
for col in cols[1:]:
    if (df[[col]].describe().loc['std',:]).get(col) ==0:
        zero_std.append(col)

In [30]:
zero_std

['cd_000']

In [20]:
## train model

import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

In [6]:
from sensor.utils.main_utils import load_numpy_array_data


In [7]:

test_file =r"C:\ineuron\Industry_ready_projects\Sensor_fault_detection\Scania_APS_Fault_detection\artifact\11_01_2022_00_33_15\data_transformation\transformed\test.npy"
train_file = r"C:\ineuron\Industry_ready_projects\Sensor_fault_detection\Scania_APS_Fault_detection\artifact\11_01_2022_00_33_15\data_transformation\transformed\train.npy"

In [8]:
test_data = load_numpy_array_data(test_file)
train_data = load_numpy_array_data(train_file)

In [19]:
x_test_data = test_data[:,:-1]
y_test_data = test_data[:,-1]
x_train_data = train_data[:,:-1]
y_train_data = train_data[:,-1]

In [21]:
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, x_train_data, y_train_data, cv=5)
scores.mean()


0.9738736680638999

In [23]:
clf.fit(x_train_data, y_train_data)

AdaBoostClassifier(n_estimators=100)

In [24]:
from sklearn.metrics import accuracy_score

y_pred =clf.predict(x_test_data)
accuracy_score(y_test_data,y_pred)

0.9806752554420257

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier(n_estimators=10)
clf2 = clf2.fit(x_train_data, y_train_data)

In [26]:
y_pred =clf2.predict(x_test_data)
accuracy_score(y_test_data,y_pred)

0.9843402932030209

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
clf3 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, \
max_depth=1, random_state=0).fit(x_train_data, y_train_data)

In [28]:
y_pred =clf3.predict(x_test_data)
accuracy_score(y_test_data,y_pred)

0.9777506293499185

In [29]:
from sklearn.ensemble import HistGradientBoostingClassifier
clf4 = HistGradientBoostingClassifier(max_iter=100).fit(x_train_data, y_train_data)

In [30]:
y_pred =clf4.predict(x_test_data)
accuracy_score(y_test_data,y_pred)

0.992966089145565

In [31]:

scores = cross_val_score(clf4, x_train_data, y_train_data, cv=5)
scores.mean()

0.9967699666472702

In [32]:
from sklearn.model_selection import GridSearchCV



In [33]:
param_grid = {'min_samples_leaf':[10,20,30],
'max_depth':[1,2,3],
'learning_rate':[0.05,0.1,0.2],
'max_iter':[50,100,120]}



In [36]:
hbc=HistGradientBoostingClassifier()

In [37]:
clf_gr = GridSearchCV(estimator=hbc,
param_grid=param_grid,
cv =5,
n_jobs=5,
verbose=1)

In [41]:
clf_gr.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__categorical_features': None,
 'estimator__early_stopping': 'auto',
 'estimator__l2_regularization': 0.0,
 'estimator__learning_rate': 0.1,
 'estimator__loss': 'log_loss',
 'estimator__max_bins': 255,
 'estimator__max_depth': None,
 'estimator__max_iter': 100,
 'estimator__max_leaf_nodes': 31,
 'estimator__min_samples_leaf': 20,
 'estimator__monotonic_cst': None,
 'estimator__n_iter_no_change': 10,
 'estimator__random_state': None,
 'estimator__scoring': 'loss',
 'estimator__tol': 1e-07,
 'estimator__validation_fraction': 0.1,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': HistGradientBoostingClassifier(),
 'n_jobs': 5,
 'param_grid': {'min_samples_leaf': [10, 20, 30],
  'max_depth': [1, 2, 3],
  'learning_rate': [0.05, 0.1, 0.2],
  'max_iter': [50, 100, 120]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 1}

In [38]:
clf_gr.fit(x_train_data,y_train_data)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=HistGradientBoostingClassifier(), n_jobs=5,
             param_grid={'learning_rate': [0.05, 0.1, 0.2],
                         'max_depth': [1, 2, 3], 'max_iter': [50, 100, 120],
                         'min_samples_leaf': [10, 20, 30]},
             verbose=1)

In [39]:
y_pred =clf_gr.predict(x_test_data)
accuracy_score(y_test_data,y_pred)

0.9883014956315711